In [8]:
import os
import csv
from countries_languages import country_to_language
import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime
from functools import reduce
from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


import requests
from bs4 import BeautifulSoup

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

countries_codes = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/countries_and_codes.csv')

#leagues value
leagues_value = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/Most Updated Edited Transfermarkt Dataset.csv')
#leagues value large 
leagues_value_large = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/most_updated_transfermarkt_dataset.csv')

from fuzzywuzzy import process

def find_close_matches_variable(this_jersey, dataset_nationality, threshold):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]

#threshold match one 
def threshold_player_match(this_jersey, dataset_nationality):
    THRESHOLD_NUM = 89
    #stops when it returns a name.
    #if it doesnt find a match keep lowering the threshold until you find a match
    #but if you get to threshold of like 50 first you stop and just abandon ship    

    # Loop until someone is found or threshold goes below 50
    while THRESHOLD_NUM >= 50:
        matches = find_close_matches_variable(this_jersey, dataset_nationality, THRESHOLD_NUM)
        if matches:
            #print(f"Player is {this_jersey}. Found matches: {matches}. threshold is {THRESHOLD_NUM}")
            return matches, THRESHOLD_NUM
            #break
        else:
            THRESHOLD_NUM -= 1

    # If threshold reaches below 50 without finding any matches
    if THRESHOLD_NUM < 50:
        return [f"No matches found even with the lowest threshold."], this_jersey #jersey was {this_jersey}


def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    #print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        #print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''
    

def check_tokens(this_jersey, match):
    jersey_tokens = this_jersey.lower().split()
    match_tokens = match.lower().split()
    
    if all(token in match_tokens for token in jersey_tokens):
        return True
    elif all(token in jersey_tokens for token in match_tokens) and len(jersey_tokens) == len(match_tokens) + 1:
        return True
    elif all(token in match_tokens for token in jersey_tokens) and len(jersey_tokens) > len(match_tokens):
        return True
    else:
        return False

def filter_multi_word_matches_by_jersey_tokens(jersey_tokens, potential_matches):
    filtered_matches = []

    for match in potential_matches:
        # Split each match into tokens
        match_tokens = match.split()

        # Count the number of tokens that start with each character in jersey_tokens
        match_token_start_chars = {token[0] for token in match_tokens}
        #print(match_token_start_chars, f"*{jersey_tokens}")


        #for token in jersey_tokens:
        #   print(token, token[0] in match_token_start_chars)

        # Check if the match contains at least one token for each character in jersey_tokens
        if all(token[0] in match_token_start_chars for token in jersey_tokens):
            filtered_matches.append(match)

    return filtered_matches

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_38101/3382385496.py:49: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [9]:
for index, row in leagues_value.iterrows():
    if row['Nationality'] in countries_codes['Country'].unique():
        0==0
    else:
        print(row['Nationality'])

In [35]:
for index, row in leagues_value.iterrows():
    if row['Nationality'] == 'China':
        leagues_value.at[index, 'Team 1 Code'] = 'CN'
    if row['Nationality'] == 'Democratic Republic of the Congo':
        leagues_value.at[index, 'Team 1 Code'] = 'DR'
    if row['Nationality'] == 'Republic of the Congo':
        leagues_value.at[index, 'Team 1 Code'] = 'CG'
    if row['Nationality'] == 'Brunei Darussalam':
        leagues_value.at[index, 'Team 1 Code'] = 'BN'
    if row['Nationality'] == 'United States Virgin Islands':
        leagues_value.at[index, 'Team 1 Code'] = 'VI'
    if row['Nationality'] == 'Turks and Caicos Islands':
        leagues_value.at[index, 'Team 1 Code'] = 'TC'
    if row['Nationality'] == 'Namibia':
        leagues_value.at[index, 'Team 1 Code'] = 'NB'
        

In [53]:
for index, row in leagues_value_large.iterrows():
    if row['Nationality'] == 'China':
        leagues_value_large.at[index, 'Team 1 Code'] = 'CN'
    if row['Nationality'] == 'Democratic Republic of the Congo':
        leagues_value_large.at[index, 'Team 1 Code'] = 'DR'
    if row['Nationality'] == 'Republic of the Congo':
        leagues_value_large.at[index, 'Team 1 Code'] = 'CG'
    if row['Nationality'] == 'Brunei Darussalam':
        leagues_value_large.at[index, 'Team 1 Code'] = 'BN'
    if row['Nationality'] == 'United States Virgin Islands':
        leagues_value_large.at[index, 'Team 1 Code'] = 'VI'
    if row['Nationality'] == 'Turks and Caicos Islands':
        leagues_value_large.at[index, 'Team 1 Code'] = 'TC'
    if row['Nationality'] == 'Namibia':
        leagues_value_large.at[index, 'Team 1 Code'] = 'NB'
    if row['Nationality'] == 'United States':
        leagues_value_large.at[index, 'Team 1 Code'] = 'US'
    if row['Nationality'] == 'Bosnia-Herzegovina':
        leagues_value_large.at[index, 'Team 1 Code'] = 'BA'
    if row['Nationality'] == 'Czech Republic':
        leagues_value_large.at[index, 'Team 1 Code'] = 'CZ'
    if row['Nationality'] == 'British Virgin Islands':
        leagues_value_large.at[index, 'Team 1 Code'] = 'BVI'
    if row['Nationality'] == 'Tahiti':
        leagues_value_large.at[index, 'Team 1 Code'] = 'TI'
    if row['Nationality'] == 'Curacao':
        leagues_value_large.at[index, 'Team 1 Code'] = 'CW'
    if row['Nationality'] == 'Korea, South':
        leagues_value_large.at[index, 'Team 1 Code'] = 'KR'
    if row['Nationality'] == 'Northern Ireland':
        leagues_value_large.at[index, 'Team 1 Code'] = 'NI'
    if row['Nationality'] == 'Neukaledonien':
        leagues_value_large.at[index, 'Team 1 Code'] = 'NC'
    if row['Nationality'] == 'Sao Tome and Principe':
        leagues_value_large.at[index, 'Team 1 Code'] = 'ST'
    if row['Nationality'] == 'Korea, North':
        leagues_value_large.at[index, 'Team 1 Code'] = 'NK'
    if row['Nationality'] == 'St. Lucia':
        leagues_value_large.at[index, 'Team 1 Code'] = 'LC'
    if row['Nationality'] == 'St. Kitts &amp; Nevis':
        leagues_value_large.at[index, 'Team 1 Code'] = 'KN'
    if row['Nationality'] == 'Saint-Martin':
        leagues_value_large.at[index, 'Team 1 Code'] = 'MF'
    if row['Nationality'] == 'Macao':
        leagues_value_large.at[index, 'Team 1 Code'] = 'MF'




In [54]:
leagues_value_large[leagues_value_large['Team 1 Code'].isnull()]['Nationality'].value_counts()

Series([], Name: count, dtype: int64)

In [55]:
countries_codes.to_csv('countries_and_codes_newest.csv')

In [56]:
leagues_value.to_csv('NEW Most Updated Edited Transfermarkt Dataset.csv')

leagues_value_large.to_csv('most_updated_transfermarkt_dataset.csv')